In [1]:
import pandas as pd
import numpy as np
import regex as re
from tqdm import tqdm

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Embedding,Input, GRU,LSTM,Conv1D,MaxPooling1D, Dense, BatchNormalization, Dropout,Activation,Attention, GlobalMaxPooling1D,Concatenate

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score , classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
# from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# 

In [6]:
# from keras.models import load_model

# import shutil

# # Source file path (in the Kaggle input directory)
# src_file = '/kaggle/input/odi/keras/odi/1/model_checkpoint (2).keras'

# # Destination file path (in the Kaggle output directory)
# dst_file = '/kaggle/working/model_checkpoint (2).keras'

# # Copy the file
# shutil.copyfile(src_file, dst_file)

# model = load_model('/kaggle/working/model_checkpoint (2).keras')
# # Compile the model (if needed)



In [7]:
import os
print(os.curdir)

.


In [8]:
df_lite = pd.read_csv('WikiAnswer_lite.csv',index_col=0)

In [9]:
# def addit(x):
#     return "<start>"+x+"<end>"

In [10]:
# df_lite=df_lite.apply(addit)

In [11]:
df_lite.head(5)

,input,output
0,Calories in a handful of strawberries?,How many calories are in a strawberry popsickles?
1,Replace water pump in 1998 taurus 3.0 24v?,How do you replace water pump on 01 ford tauru...
2,How important was britain in ww2?,What was the meanig of the battle of britain?
3,Definition of tally 5.4?,Difference bw tally 8.1 7.2?
4,Structural feature of glycerol important in fa...,Where does fats change into fatty acids and gl...


In [13]:
long_sentences = [sentence for sentence in df_lite['input'] if len(sentence.split()) > 42]
long_sentences

['Which formulas can be used to find the surface area of a regular pyramid with a square base where the perimeter of the base is equal to p s is the slant height BA is the base area and LA is the latera?',
 "You 8w 2 day pregnant and had an us at 7w 4 day all great but last night you wiped and there's pink and a little red discharge not much what could cause this you had a rough bm could this cause that?"]

In [15]:
x = df_lite['input']
y = df_lite['output']

In [16]:
max_input = max(len(i.split()) for i in x)
max_output = max(len(i.split()) for i in y)

In [17]:
print("Max input_length:",max_input)
print("Max output_length:",max_output)

Max input_length: 34
Max output_length: 39


In [18]:
## Adding addtional length
max_input,max_output=40,40

In [19]:
tokenizer_in = Tokenizer()
tokenizer_in.fit_on_texts(x)
x_tok = tokenizer_in.texts_to_sequences(x)
tokenizer_out = Tokenizer()
tokenizer_out.fit_on_texts(y)
y_tok = tokenizer_out.texts_to_sequences(y)

In [20]:
len(tokenizer_in.word_index)

2947

In [21]:
embeddings_index = {}
with open('glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [22]:
# Create embedding matrix
embedding_matrix = np.zeros((len(tokenizer_in.word_index) + 1, 200))
for word, i in tokenizer_in.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
# Create embedding matrix
embedding_matrix_out = np.zeros((len(tokenizer_out.word_index) + 1, 200))
for word, i in tokenizer_out.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_out[i] = embedding_vector

In [24]:
x_pad = pad_sequences(x_tok, maxlen=max_input,padding='post') 
y_pad = pad_sequences(y_tok, maxlen=max_output,padding='post') 

In [25]:
# from gensim.scripts.glove2word2vec import glove2word2vec
# from gensim.models import KeyedVectors

# # Convert GloVe file to Word2Vec format (if needed)
# # glove2word2vec("/kaggle/input/glove/other/wordglove/1/glove.6B.200d.txt", "glove.6B.200d.word2vec")

# # Load the embeddings
# model = KeyedVectors.load_word2vec_format("glove.6B.200d.word2vec", binary=False)

In [26]:
# def get_embeddings(sent):
#     vec = np.array([model[word] if word in model else np.zeros((200)) for word in sent])
#     vec = vec.sum(axis=0)
#     return vec

# input_vec = np.array([get_embeddings(sent) for sent in [word_tokenize(sent) for sent in x]])
# output_vec = np.array([get_embeddings(sent) for sent in [word_tokenize(sent) for sent in y]])

In [28]:
max_input_length = 40
max_output_length = 40
vocab_in_size = len(tokenizer_in.word_index) + 1
vocab_out_size = len(tokenizer_out.word_index) + 1
embedding_dim = 200
latent_dim = 64

In [29]:
encoder_embedding = Embedding(
    vocab_in_size,
    200,
    trainable=False,
)
encoder_embedding.build((1,))
encoder_embedding.set_weights([embedding_matrix])

In [30]:
decoder_embedding = Embedding(
    vocab_out_size,
    200,
    trainable=False,
)
decoder_embedding.build((1,))
decoder_embedding.set_weights([embedding_matrix_out])

In [31]:
from keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate, BatchNormalization, Dropout
from keras.models import Model

# Define the input and output lengths and vocab sizes


# Encoder
encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = encoder_embedding(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
attention_layer = Attention()([decoder_outputs, encoder_outputs])
decoder_concat = Concatenate(axis=-1)([decoder_outputs, attention_layer])
decoder_bn = BatchNormalization()
decoder_outputs = decoder_bn(decoder_concat)
decoder_dropout = Dropout(0.5)
decoder_outputs = decoder_dropout(decoder_outputs)
decoder_dense = Dense(vocab_out_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model_main = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model_main.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 40)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 40, 200)           │         589,600 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, None, 200)         │         608,200 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 40, 64), (None,   │          67,840 │ embedding[0][0]            │
│                               │ 64), (None, 64)]          │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, None, 64), (None, │          67,840 │ embedding_1[0][0],         │
│                               │ 64), (None, 64)]          │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attention (Attention)         │ (None, None, 64)          │               0 │ lstm_1[0][0], lstm[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, None, 128)         │               0 │ lstm_1[0][0],              │
│                               │                           │                 │ attention[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, None, 128)         │             512 │ concatenate[0][0]          │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, None, 128)         │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 3041)        │         392,289 │ dropout[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,726,281 (6.59 MB)

 Trainable params: 1,726,025 (6.58 MB)

 Non-trainable params: 256 (1.00 KB)

In [32]:
model_main.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# from keras.callbacks import ModelCheckpoint
# filepath = 'model_checkpoint.keras'

# from keras.callbacks import Callback
# from IPython.display import FileLink

# # Define a custom callback to download the model after each epoch
# class DownloadModelCallback(Callback):
#     def __init__(self, file_path):
#         self.file_path = file_path

#     def on_epoch_end(self, epoch, logs=None):
#         # Display a link to download the model file
#         display(FileLink(self.file_path))

# # Define the file path for saving the model
# # Create a DownloadModelCallback to download the model after each epoch
# download_callback = DownloadModelCallback(filepath)
# checkpoint = ModelCheckpoint(filepath)
# # Train the model with the callback
# # model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32, callbacks=[download_callback])


In [ ]:
# model_main.fit([input_vec,input_vec], output_vec, batch_size=32, epochs=35,verbose=True, validation_split=0.2, callbacks=[download_callback,checkpoint])

In [ ]:
# model_main.save('model_batch.keras')

In [32]:
from keras.models import load_model
model_main = load_model('model_batch.keras')

In [46]:
from keras.preprocessing.sequence import pad_sequences

# Custom input
custom_input_enc = tokenizer_in.texts_to_sequences(['How many water need to be drank?'])
custom_input_enc = pad_sequences(custom_input_enc, maxlen=max_input,padding='post')

# Decoder input for inference (start with a start token)
decoder_input = np.zeros((max_output,))

# Decode predictions iteratively
decoded_sentence = []
for i in range(1, max_output-1):
    predictions = model_main.predict([custom_input_enc, np.reshape(decoder_input[:-1],newshape=(1,-1))],verbose=False)
    predicted_id = np.argmax(predictions[0, i-1, :])
    # if tokenizer_out.index_word.get(predicted_id) == 'end':
    #     break  # Stop if end token is predicted
    decoded_sentence.append(tokenizer_out.index_word.get(predicted_id, ''))    
    # Update decoder input for next step
    decoder_input[i] = predicted_id
    # print(decoded_sentence[i-1])
# Convert the decoded sentence to text
decoded_sentence = ' '.join(decoded_sentence)
print(decoded_sentence)
print('INPUT:',df_lite['input'][110])
print("OUTPUT:",df_lite['output'][110])

like have very few                                  
INPUT: What is the period in greek history associated with alexander the great?
OUTPUT: How did alexander of macedon influence history?
